In [3]:
!pip install ndjson

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import json
import csv
import os
import ndjson
import pandas as pd

path = "C:\\github\\solliancenet\\microsoft-postgres-docs-project\\artifacts\\data"

In [5]:
def parse_string(input):

    return input.replace("\n", " ").replace("\r", " ").replace("\"", "\\\"")

def make_json(csvFilePath, jsonFilePath, keyColumn = None):
     
    # create a dictionary
    data = []

    count = 0

    jsonfile = open(jsonFilePath, 'w')
     
    # Open a csv reader called DictReader
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
         
        # Convert each row into a dictionary 
        # and add it to data
        for row in csvReader:

            max_listing_id = 7500000

            if (csvFilePath.endswith("listings.csv")):
                if (int(row['id']) > max_listing_id):
                    continue

            if (csvFilePath.endswith("review.csv") or csvFilePath.endswith("calendar.csv")):
                if (int(row['listing_id']) > max_listing_id):
                    continue

            for key in row.keys():
                try:
                    row['key'] = row['key'].encode('ascii', 'replace').decode('utf-8')
                except:
                    pass

            #convert some values to date
            dates = {"date"}

            for key in dates:
                if (key in row):
                    try:
                        row[key] = pd.to_datetime(row[key]).strftime('%Y-%m-%d')
                    except:
                        row[key] = pd.to_datetime('1900-01-01').strftime('%Y-%m-%d')
            
            #convert some values to integers
            integers = {"bathrooms", "bedrooms", "beds", "id", "listing_id"}

            for key in integers:
                if (key in row):
                    try:
                        row[key] = int(row[key])
                    except:
                        row[key] = 0

            if ('city' in row):
                row['city'] = row['city'].strip()

            if ('host_verifications' in row ):
                row['host_verifications'] = json.loads(row['host_verifications'].replace('"', '').replace("'", '"'))

            if ('amenities' in row ):
                vals = row['amenities'].replace('"', '').replace("'", '"').replace("{","").replace("}","").split(",")
                row['amenities'] = vals

            floats = {"latitude", "longitude", "price"}

            for key in floats:
                if (key in row):
                    try:
                        row[key] = float(row[key].replace("$", ""))
                    except:
                        row[key] = -1
                    
            #convert some values to arrays

            #if ('amenities' in row ):
                #remove unicode chars
            #    row['amenities'] = row['amenities'].encode('ascii', 'replace').decode('utf-8')
            
            #if ( 'neighborhood_overview' in row ):
            #    row['neighborhood_overview'] = row['neighborhood_overview'].encode('ascii', 'replace').decode('utf-8')
            
            #if ('comments' in row ):
            #    row['comments'] = row['comments'].encode('ascii', 'replace').decode('utf-8')

            #if ('space' in row ):
            #    row['space'] = row['space'].encode('ascii', 'replace').decode('utf-8')

            j = json.dumps(row)

            if ( "\\u" not in j):
                jsonfile.write(j)
                jsonfile.write('\n')
            else:
                pass
                #print(j)

            continue
             
            # Assuming a column named 'No' to
            # be the primary key
            try:
                if ('amenities' in row ):
                    row['amenities'] = parse_string(row['amenities'])
            except:
                pass

            try:
                if ( 'neighborhood_overview' in row ):
                    if ('\"Main' in row['neighborhood_overview']):
                        print(row['neighborhood_overview'])
                    
                    row['neighborhood_overview'] = parse_string(row['neighborhood_overview'])
            except:
                pass
            
            try:
                row['comments'] = parse_string(row['comments'])
            except:
                pass
            
            data.append(row)

    jsonfile.close()

    #replace all the text in the generated file
    with open(jsonFilePath, 'r', encoding='utf-8') as file :
        filedata = file.read()
        filedata = filedata.replace("\\n", "")
        filedata = filedata.replace("\\t", "")
        filedata = filedata.replace("a\\\\lovely", "a lovely")
        filedata = filedata.replace("list\\\\map", "list map")
        filedata = filedata.replace("\\\\We", "We")
        filedata = filedata.replace("cottage\\\\and", "cottage and")
        filedata = filedata.replace("s\\\\chose", "chose")
        filedata = filedata.replace("\\\\", "/")
        filedata = filedata.replace("\\\"", "\\\\\"")
        
    file.close()

    #write the file out again
    with open(jsonFilePath, 'w') as file:
        file.write(filedata)

    file.close()
 
    # Open a json writer, and use the json.dumps() 
    # function to dump data
    #with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
    #    j = ndjson.dumps(data)
    #    j = j.replace("\\\"", "\\\\\"")
    #    jsonf.write(j)

In [6]:
#get each csv file in the directory
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        csvFilePath = os.path.join(path, filename)
        jsonFilePath = os.path.join(path, filename.replace(".csv", ".json"))

        key = "id"

        if (filename.startswith("calendar")):
            key = "listing_id"

        if (filename.startswith("listings")):
            key = "id"

        if (filename.startswith("reviews")):
            key = "listing_id"

        make_json(csvFilePath, jsonFilePath, key)

        continue
    else:
        continue

In [ ]:
#load the json file

calendar_json = json.loads(open(f"{path}\\calendar.json", encoding='utf-8').read())
calendar_df = pd.read_csv(f"{path}\\calendar.csv", encoding='utf-8')

listings_json = json.loads(open(f"{path}\\listings.json", encoding='utf-8').read())
listings_df = pd.read_csv(f"{path}\\listings.csv", encoding='utf-8')

reviews_json = json.loads(open(f"{path}\\reviews.json", encoding='utf-8').read())
reviews_df = pd.read_csv(f"{path}\\reviews.csv", encoding='utf-8')

In [ ]:
max_listings = 1000
listing_count = 0

listings_export = []

#merge the data...
for listingKey in listings_json.keys():

    if (listing_count > max_listings):
        break

    listing_count += 1

    listing = listings_json[listingKey]

    listing_calendar = calendar_df[calendar_df['listing_id'] == int(listingKey)]

    listing['calendar'] = []

    count = len(listing_calendar)

    print(f"Adding {count} calendar to listing {listingKey}")

    #add the calendar data from dataframes
    for index, row in listing_calendar.iterrows():
        item = row.to_dict()
        item.pop('listing_id', None)
        listing['calendar'].append(item)
    
    listing_reviews = reviews_df[reviews_df['listing_id'] == int(listingKey)]

    count = len(listing_reviews)

    print(f"Adding {count} reviews to listing {listingKey}")

    #add the reviews data
    listing['reviews'] = []

    for index, row in listing_reviews.iterrows():
        item = row.to_dict()
        try:
            item['comments'] = item['comments'].replace("\n", " ").replace("\r", " ").replace("\"", "")
            item.pop('listing_id', None)
            listing['reviews'].append(item)
        except Exception as e:
            print(f"Error adding review for listing {listingKey}: {e}")

    listing['amenities'] = None
    listings_export.append(listing)

#save to a new file
with open(f"{path}\\listings_merged.json", 'w', encoding='utf-8') as jsonf:
    jsonf.write(json.dumps(listings_export, indent=4).replace(" NaN", " null"))